# **FIND CLAPS IN VIDEO**

In [9]:
import os
from clap_detection_methods import process_videos_in_directory, load_accelerometer_data, plot_accelerometer_data_interval, normalize_data, find_peaks_in_interval

In [ ]:
directory_path = os.path.join(os.path.dirname(os.getcwd()), "Data\\videos\\FK")
clap_results = process_videos_in_directory(directory_path, num_segments= 4)

In [ ]:
import pandas as pd

df_clap_results = pd.DataFrame(clap_results)
df_clap_results

# **FIND PEAKS IN ACCELERATION DATA**

In [ ]:
acc_data_path1 = r"C:\Users\niki\Desktop\Step-counter\Data\accelerometer_data\STM2D48233516 (2025-01-16)RAW.csv"
acc_data_path2 = r"C:\Users\niki\Desktop\Step-counter\Data\accelerometer_data\STM2D48232122 (2025-01-16)RAW.csv"
sampling_frequency = 256
cleaned_data, time_seconds = load_accelerometer_data(acc_data_path1, sampling_frequency)
plot_accelerometer_data_interval(cleaned_data, time_seconds, title_suffix="Full Duration", plot_each_axis=False)

In [ ]:
start_time = 3000
end_time = 3600
plot_accelerometer_data_interval(cleaned_data, time_seconds, start_time=start_time, end_time=end_time, title_suffix="Full Duration", plot_each_axis=False)

In [ ]:
start_time = 3420
end_time = 3460
plot_accelerometer_data_interval(cleaned_data, time_seconds, start_time=start_time, end_time=end_time, title_suffix="Full Duration", plot_each_axis=False)

In [ ]:
# Find the peaks in the interval
normalized_data = normalize_data(cleaned_data)
first_half_time, second_half_time, index_first_clap, index_last_clap = find_peaks_in_interval(normalized_data, time_seconds, start_time, end_time)

# Print the results
print("Peaks in the interval:")
print(f"First Half Peak - Time: {first_half_time:.2f}")
print(f"Second Half Peak - Time: {second_half_time:.2f}")
print(f"First Half Peak - Index: {index_first_clap}")
print(f"Second Half Peak - Index: {index_last_clap}")
duration = second_half_time - first_half_time
print(f"Total Duration: {duration:.3f}s")

In [ ]:
df_clap_results